<a href="https://colab.research.google.com/github/KWeselski/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
import os
import datetime
import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [0]:
%load_ext tensorboard

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

In [0]:
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  logdir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  model.fit(X_train,
             y_train,
             batch_size=params_fit.get('batch_size',128),
             epochs = params_fit.get('epochs', 5),
             verbose = params_fit.get('verbose', 1),
             validation_data = params_fit.get('validation_data', (X_train, y_train)),
             callbacks =[tensorboard_callback],
  )
  return model

In [0]:
def predict(model_trained, X_test, y_test,scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)
  return scoring(y_test_norm, y_pred)

def train_and_predict(model):
  model_trained = train_model(model,X_train,y_train)
  return predict(model_trained,X_test,y_test)

In [0]:
def get_cnn_v3(input_shape, num_classes):
 return Sequential([
      Conv2D(64,(3,3),activation='relu',input_shape=input_shape),
      Conv2D(64,(3,3),activation='relu'),
      MaxPooling2D(),
      Dropout(0.5),
      Conv2D(128,(3,3),activation='relu',padding='same'),
      Conv2D(128,(3,3),activation='relu'),
      MaxPooling2D(),
      Dropout(0.3),
      Conv2D(256,(3,3),activation='relu',padding='same'),
      Conv2D(256,(3,3),activation='relu'),
      Dropout(0.3),
      Flatten(),
      Dense(1024, activation='relu'),
      Dense(512, activation='relu'),
      Dense(num_classes, activation='softmax'),
  ])

In [0]:
X_train_gray = color.rgb2gray(X_train).reshape(-1,32,32,1)
X_test_gray = color.rgb2gray(X_test).reshape(-1,32,32,1)

In [12]:
model = get_cnn_v3((32,32,1), num_classes)
model_trained = train_model(model, X_train_gray, y_train, params_fit={'epochs':15})
predict(model_trained,X_test_gray,y_test)

Epoch 1/15
 22/272 [=>............................] - ETA: 4s - loss: 3.6950 - accuracy: 0.0597

KeyboardInterrupt: ignored

In [0]:
def get_model(params):
 return Sequential([
      Conv2D(64,(3,3),activation='relu',input_shape=input_shape),
      Conv2D(64,(3,3),activation='relu'),
      MaxPooling2D(),
      Dropout(params['dropout_cnn_block_one']),
      Conv2D(128,(3,3),activation='relu',padding='same'),
      Conv2D(128,(3,3),activation='relu'),
      MaxPooling2D(),
      Dropout(params['dropout_cnn_block_two']),
      Conv2D(256,(3,3),activation='relu',padding='same'),
      Conv2D(256,(3,3),activation='relu'),
      Dropout(params['dropout_cnn_block_three']),
      Flatten(),
      Dense(1024, activation='relu'),
      Dropout(params['dropout_dense_block_one'])
      Dense(512, activation='relu'),
      Dense(num_classes, activation='softmax'),
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  logdir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size',128)),
      epochs = 5,
      verbose = 0,
  )
  score = model.evaluate(X_test,y_test,verbose=0)
  accuracy = score[1]
  print(params, 'accuarcy={}'.format(accuracy))
  return {'loss': -accuracy, 'status': STATUS_OK, 'model':model}

In [27]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one':hp.uniform('dropout_cnn_block_one', 0.3,0.5 ),
    'dropout_cnn_block_two':hp.uniform('dropout_cnn_block_two', 0.3,0.5 ),
    'dropout_cnn_block_three':hp.uniform('dropout_cnn_block_three', 0.3,0.5 ),
    'dropout_dense_block_one':hp.uniform('dropout_dense_block_one', 0.3,0.7 ),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 100.0, 'dropout_cnn_block_one': 0.39344930351246576, 'dropout_cnn_block_three': 0.3684289528835194, 'dropout_cnn_block_two': 0.30005926376067377, 'dropout_dense_block_one': 0.34114016479953607}
accuarcy=0.958730161190033
{'batch_size': 200.0, 'dropout_cnn_block_one': 0.4780008394748525, 'dropout_cnn_block_three': 0.4451079422738383, 'dropout_cnn_block_two': 0.3221330807775355, 'dropout_dense_block_one': 0.41319340455846504}
accuarcy=0.9666666388511658
{'batch_size': 200.0, 'dropout_cnn_block_one': 0.3025741494097407, 'dropout_cnn_block_three': 0.47902186214311837, 'dropout_cnn_block_two': 0.31250499689400396, 'dropout_dense_block_one': 0.6054885202942248}
accuarcy=0.9609977602958679
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.4387599801811823, 'dropout_cnn_block_three': 0.43492592467570146, 'dropout_cnn_block_two': 0.31960118840123697, 'dropout_dense_block_one': 0.33589318781507105}
accuarcy=0.9687075018882751
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.3421936